In [55]:
"""
Testing a 10 month SMA strategy using month end data. The strategy is to sell when the S&P500 goes below the 10 month
SMA and buy when the S&P500 rises above the 10 month SMA. Buy and sell signals are only made at month end. 

Month-end data from January 1950 was downloaded from Yahoo Finance.

"""

import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv
import plotly.express as px
import panel as pn
import hvplot
import hvplot.pandas

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from datetime import datetime, timedelta


pn.extension()

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

%matplotlib inline

In [56]:
#Read in month end data from 1950 from csv

csvpath = Path(f"C:/Users/costa/Desktop/byte/rice/ru-hou-fin-pt-03-2020-u-c/Project2_draft/SPY_month_end.csv",
                  index_col="date", infer_datetime_format=True, parse_dates=True)
stock_data_df = pd.read_csv(csvpath)
#stock_data_df['Date'] = pd.to_datetime(stock_data_df['Date'] )
stock_data_df.set_index('Date',inplace=True) 

#Change Monthly Close column to 'Close'
stock_data_df.rename(columns={'Monthly Close': 'Close'},inplace=True)


stock_data_df.head()



,Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [57]:
#Clean data

# Drop nulls
stock_data_df.dropna(inplace=True)

# drop duplicates
stock_data_df.drop_duplicates(inplace=True)

#count nulls 
stock_data_df.isnull().sum()

stock_data_df.head()



,Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [58]:

#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['Close'].to_frame()

signals_df.head()


,Close
Date,
01/03/50,17.05
02/01/50,17.22
03/01/50,17.29
04/03/50,17.96
05/01/50,18.78


In [59]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 1
long_window = 10   #Use 10 here to represent 10 month end closing prices

#Set names of the windows
short_name = "SMA1"
long_name = "SMA10"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.head(10)


,Close,SMA1,SMA10,Signal,Buy/Sell
Date,,,,,
01/03/50,17.05,17.05,NaN,0.0,NaN
02/01/50,17.22,17.22,NaN,0.0,0.0
03/01/50,17.29,17.29,NaN,0.0,0.0
04/03/50,17.96,17.96,NaN,0.0,0.0
05/01/50,18.78,18.78,NaN,0.0,0.0
06/01/50,17.69,17.69,NaN,0.0,0.0
07/03/50,17.84,17.84,NaN,0.0,0.0
08/01/50,18.42,18.42,NaN,0.0,0.0
09/01/50,19.45,19.45,NaN,0.0,0.0


In [60]:
len(signals_df)

832

In [61]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA1', 'SMA10']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [62]:
signals_df.head()

,Close,SMA1,SMA10,Signal,Buy/Sell
Date,,,,,
01/03/50,17.05,17.05,NaN,0.0,NaN
02/01/50,17.22,17.22,NaN,0.0,0.0
03/01/50,17.29,17.29,NaN,0.0,0.0
04/03/50,17.96,17.96,NaN,0.0,0.0
05/01/50,18.78,18.78,NaN,0.0,0.0


In [64]:

# Add share and cash columns

signals_df["Shares"] = float(0)
signals_df["Portfolio Cash"] = float(100000)

#Set up starting cash variable for sma strategy and buy and hold strategy

cash = 100000
buy_and_hold_cash = 100000


#Iterate through each row, use combinations of signal and buy/sell points to set shares and portfolio cash for each row.

for index, row in signals_df.iterrows():
    
    #If at a buy point, calculate the number of shares using total cash available and set the cash to zero
    
    if row['Buy/Sell'] == 1.0:
       
        shares = cash/row['Close']
        row['Shares'] = shares
        row['Portfolio Cash']= 0 
       
    #If at sell point, calculate the cash generated from the sale (shares X closing price) and set the shares to zero, then
    #update the cash variable with the new cash balance'''
    
    elif row['Buy/Sell'] == -1.0:
        row['Portfolio Cash']= shares*row['Close']
        cash = row['Portfolio Cash']
        row['Shares']= 0
        
    #If we are not at a buy or sell point, look to signal column, if the signal is buy, then set cash to zero and update shares.
    # If the signal is sell, update the cash column using cash variable
    else:
     
        row['Buy/Sell'] == 0
        if row['Signal']== 1:
            row['Portfolio Cash'] = 0
            row["Shares"]= shares
        elif row['Signal'] == 0:
            row['Portfolio Cash'] = cash
        
# Add a Portfolio Total column
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + (signals_df['Shares']*signals_df['Close'])

# Calculate how many shares a buy and hold investor could buy at the start of the comparison. Use end of long window as start. 
buy_and_hold_shares = buy_and_hold_cash/signals_df['Close'].iloc[long_window-1]

# Add a buy and hold total column for comparison, set the first(long window) to 100000 so both portfolios are at 100000 at the
#end of the long window calculation
signals_df["Buy&Hold Total"]= buy_and_hold_shares * signals_df['Close']
signals_df["Buy&Hold Total"][0:long_window-1] = buy_and_hold_cash      
        
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()  
signals_df['Buy&Hold Daily Returns']= signals_df['Buy&Hold Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cum Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1
signals_df['Buy&Hold Cum Returns'] = (1 + signals_df['Buy&Hold Daily Returns']).cumprod() - 1

# Print the DataFrame
pd.set_option("display.max_rows", None, "display.max_columns", None)
signals_df.tail(1000)

,Close,SMA1,SMA10,Signal,Buy/Sell,Shares,Portfolio Cash,Portfolio Total,Buy&Hold Total,Portfolio Daily Returns,Buy&Hold Daily Returns,Portfolio Cum Returns,Buy&Hold Cum Returns
Date,,,,,,,,,,,,,
01/03/50,17.05,17.05,NaN,0.0,NaN,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,NaN,NaN,NaN,NaN
02/01/50,17.22,17.22,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
03/01/50,17.29,17.29,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
04/03/50,17.96,17.96,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
05/01/50,18.78,18.78,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
06/01/50,17.69,17.69,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
07/03/50,17.84,17.84,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
08/01/50,18.42,18.42,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000
09/01/50,19.45,19.45,NaN,0.0,0.0,0.000000,1.000000e+05,1.000000e+05,1.000000e+05,0.000000e+00,0.000000,0.000000,0.000000


In [37]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Buy/Sell'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Buy/Sell'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [38]:
#Plot the SMA Strategy vs the Buy & Hold strategy

signals_df[["Buy&Hold Total", "Portfolio Total"]].hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [39]:
#Prepare evaluation metrics dataframe

# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest', 'Buy&Hold']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` 
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [40]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'][0] = signals_df['Portfolio Cum Returns'][-1]
portfolio_evaluation_df.loc['Cumulative Returns'][1] = signals_df['Buy&Hold Cum Returns'][-1]

portfolio_evaluation_df



,Backtest,Buy&Hold
Annual Return,NaN,NaN
Cumulative Returns,100.438,155.463
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [41]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'][0] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return'][1] = (signals_df['Buy&Hold Daily Returns'].mean() * 252)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,1.51932,1.75483
Cumulative Returns,100.438,155.463
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [42]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'][0] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility'][1] = (signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,1.51932,1.75483
Cumulative Returns,100.438,155.463
Annual Volatility,0.47677,0.657733
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [43]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'][0] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df.loc['Sharpe Ratio'][1] = (
    signals_df['Buy&Hold Daily Returns'].mean() * 252) / (
    signals_df['Buy&Hold Daily Returns'].std() * np.sqrt(252)
)


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,1.51932,1.75483
Cumulative Returns,100.438,155.463
Annual Volatility,0.47677,0.657733
Sharpe Ratio,3.18669,2.668
Sortino Ratio,NaN,NaN


In [44]:
# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

sortino_ratio_df2 = signals_df[['Buy&Hold Daily Returns']].copy()
sortino_ratio_df2.loc[:,'Downside Returns1'] = 0

target = 0
mask = sortino_ratio_df2['Buy&Hold Daily Returns'] < target
sortino_ratio_df2.loc[mask, 'Downside Returns1'] = sortino_ratio_df2['Buy&Hold Daily Returns']**2


portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,1.51932,1.75483
Cumulative Returns,100.438,155.463
Annual Volatility,0.47677,0.657733
Sharpe Ratio,3.18669,2.668
Sortino Ratio,NaN,NaN


In [45]:
# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][0] = sortino_ratio

down_stdev = np.sqrt(sortino_ratio_df2['Downside Returns1'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df2['Buy&Hold Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'][1] = sortino_ratio





portfolio_evaluation_df

,Backtest,Buy&Hold
Annual Return,1.51932,1.75483
Cumulative Returns,100.438,155.463
Annual Volatility,0.47677,0.657733
Sharpe Ratio,3.18669,2.668
Sortino Ratio,5.03662,4.05713


In [46]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [47]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Buy/Sell'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Shares']*row['Close']
        share_size = row['Shares']
        entry_share_price = row['Close']

    elif row['Buy/Sell'] == -1:
        exit_date = index
        exit_portfolio_holding = row['Portfolio Total']
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'SPY',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df
                

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,SPY,10/02/50,03/02/53,5120.327701,19.53,25.29,1.000000e+05,1.294931e+05,2.949309e+04
1,SPY,01/04/54,09/04/56,4965.225750,26.08,45.35,1.294931e+05,2.251730e+05,9.567990e+04
2,SPY,05/01/57,08/01/57,4747.480239,47.43,45.22,2.251730e+05,2.146811e+05,-1.049193e+04
3,SPY,04/01/58,09/01/59,4942.013270,43.44,56.88,2.146811e+05,2.811017e+05,6.642066e+04
4,SPY,11/02/59,01/04/60,4823.296410,58.28,55.61,2.811017e+05,2.682235e+05,-1.287820e+04
5,SPY,06/01/60,07/01/60,4712.289412,56.92,55.51,2.682235e+05,2.615792e+05,-6.644328e+03
6,SPY,08/01/60,09/01/60,4592.331202,56.96,53.52,2.615792e+05,2.457816e+05,-1.579762e+04
7,SPY,11/01/60,04/02/62,4425.307273,55.54,65.24,2.457816e+05,2.887070e+05,4.292548e+04
8,SPY,11/01/62,06/01/65,4637.119282,62.26,84.12,2.887070e+05,3.900745e+05,1.013674e+05
9,SPY,08/02/65,03/01/66,4474.870644,87.17,89.23,3.900745e+05,3.992927e+05,9.218234e+03


In [51]:
#Create plots

price_df = signals_df[['Close', 'SMA1', 'SMA10']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [52]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest,Buy&Hold]

In [53]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

In [478]:
#Build dashboard with panel

# Create rows
price_chart_row = pn.Row(price_chart)
portfolio_evaluation_row = pn.Row(portfolio_evaluation_table)
trade_evaluation_row = pn.Row(trade_evaluation_table)

# Create columns
portfolio_column = pn.Column('# Portfolio Evaluation Metrics', price_chart_row, portfolio_evaluation_row)
trade_column = pn.Column('# Trade Evaluation Metrics', trade_evaluation_row)

# Create tabs
trading_dashboard = pn.Tabs(
    ("Portfolio Metrics", portfolio_column),
    ("Trade Metrics", trade_column)
)

In [449]:
trading_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(NdOverlay)
        [2] Row
            [0] HoloViews(Table)
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(Table)